Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [50]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [51]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [52]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [53]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [54]:
# Describe input, operations and output
batch_size = 128
num_hidden_units = 1024

graph = tf.Graph()
with graph.as_default(): # this means ops defined in this block will be added to this graph

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_units]))
  hidden_biases = tf.Variable(tf.zeros([num_hidden_units]))
    
  output_weights = tf.Variable(
    tf.truncated_normal([num_hidden_units, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))    
  
  hidden_output = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  output = tf.matmul(hidden_output, output_weights) + output_biases
  

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels))
  loss = tf.reduce_mean(loss + 0.001 * tf.nn.l2_loss(output_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output)
  valid_legits = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_legits, output_weights) + output_biases)
  test_legits = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_legits, output_weights) + output_biases)

In [55]:
# execute
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 337.344543
Minibatch accuracy: 6.2%
Validation accuracy: 29.7%
Minibatch loss at step 500: 11.879780
Minibatch accuracy: 85.2%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 5.548827
Minibatch accuracy: 85.2%
Validation accuracy: 78.1%
Minibatch loss at step 1500: 6.152618
Minibatch accuracy: 80.5%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 8.537381
Minibatch accuracy: 83.6%
Validation accuracy: 78.7%
Minibatch loss at step 2500: 3.118323
Minibatch accuracy: 82.8%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 3.988890
Minibatch accuracy: 77.3%
Validation accuracy: 80.3%
Test accuracy: 86.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [56]:
batch_size = 20

num_hidden_units = 1024

graph = tf.Graph()
with graph.as_default(): # this means ops defined in this block will be added to this graph

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_units]))
  hidden_biases = tf.Variable(tf.zeros([num_hidden_units]))
    
  output_weights = tf.Variable(
    tf.truncated_normal([num_hidden_units, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))    
  
  hidden_output = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  output = tf.matmul(hidden_output, output_weights) + output_biases
  

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels))
  loss = tf.reduce_mean(loss + 0.001 * tf.nn.l2_loss(output_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output)
  valid_legits = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_legits, output_weights) + output_biases)
  test_legits = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_legits, output_weights) + output_biases)

# execute
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 360.780243
Minibatch accuracy: 5.0%
Validation accuracy: 26.2%
Minibatch loss at step 500: 106.125687
Minibatch accuracy: 60.0%
Validation accuracy: 69.5%
Minibatch loss at step 1000: 35.531425
Minibatch accuracy: 65.0%
Validation accuracy: 63.3%
Minibatch loss at step 1500: 14.504365
Minibatch accuracy: 50.0%
Validation accuracy: 69.7%
Minibatch loss at step 2000: 61.254883
Minibatch accuracy: 70.0%
Validation accuracy: 70.5%
Minibatch loss at step 2500: 34.127575
Minibatch accuracy: 80.0%
Validation accuracy: 69.8%
Minibatch loss at step 3000: 3.215498
Minibatch accuracy: 65.0%
Validation accuracy: 68.4%
Test accuracy: 74.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [61]:
batch_size = 128
num_hidden_units = 1024

graph = tf.Graph()
with graph.as_default(): # this means ops defined in this block will be added to this graph

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_units]))
  hidden_biases = tf.Variable(tf.zeros([num_hidden_units]))
    
  output_weights = tf.Variable(
    tf.truncated_normal([num_hidden_units, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))    
  
  keep_prob = tf.placeholder(tf.float32)
  hidden_output = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  hidden_output = tf.nn.dropout(hidden_output, keep_prob)
  output = tf.matmul(hidden_output, output_weights) + output_biases
  output = tf.nn.dropout(output, keep_prob)

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels))
  loss = tf.reduce_mean(loss + 0.001 * tf.nn.l2_loss(output_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output)
  valid_legits = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_legits, output_weights) + output_biases)
  test_legits = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_legits, output_weights) + output_biases)

In [62]:
# execute
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 1.0}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 274.291443
Minibatch accuracy: 10.9%
Validation accuracy: 31.4%
Minibatch loss at step 500: 12.427366
Minibatch accuracy: 82.0%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 6.706803
Minibatch accuracy: 89.8%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 6.531335
Minibatch accuracy: 82.0%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 8.572058
Minibatch accuracy: 78.1%
Validation accuracy: 78.6%
Minibatch loss at step 2500: 4.800009
Minibatch accuracy: 76.6%
Validation accuracy: 80.2%
Minibatch loss at step 3000: 4.945078
Minibatch accuracy: 76.6%
Validation accuracy: 82.1%
Test accuracy: 88.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [97]:
batch_size = 128
num_hidden_units = 1024
num_hidden_units2 = 512
num_hidden_units3 = 256

graph = tf.Graph()
with graph.as_default(): # this means ops defined in this block will be added to this graph

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_units], stddev=np.sqrt(2.0 / (image_size * image_size))))
  hidden_biases = tf.Variable(tf.zeros([num_hidden_units]))

  hidden_weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_units, num_hidden_units2], stddev=np.sqrt(2.0 / num_hidden_units)))
  hidden_biases2 = tf.Variable(tf.zeros([num_hidden_units2]))

  hidden_weights3 = tf.Variable(
    tf.truncated_normal([num_hidden_units2, num_hidden_units3], stddev=np.sqrt(2.0 / num_hidden_units2)))
  hidden_biases3 = tf.Variable(tf.zeros([num_hidden_units3]))
    
  output_weights = tf.Variable(
    tf.truncated_normal([num_hidden_units3, num_labels], stddev=np.sqrt(2.0 / num_hidden_units3)))
  output_biases = tf.Variable(tf.zeros([num_labels]))    
  
  keep_prob = tf.placeholder(tf.float32)
  hidden_output = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  hidden_output = tf.nn.dropout(hidden_output, keep_prob)
    
  hidden_output2 = tf.nn.relu(tf.matmul(hidden_output, hidden_weights2) + hidden_biases2)
  hidden_output2 = tf.nn.dropout(hidden_output2, keep_prob)
    
  hidden_output3 = tf.nn.relu(tf.matmul(hidden_output2, hidden_weights3) + hidden_biases3)
  hidden_output3 = tf.nn.dropout(hidden_output3, keep_prob)
    
  output = tf.matmul(hidden_output3, output_weights) + output_biases
  output = tf.nn.dropout(output, keep_prob)

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, tf_train_labels) + 0.001 * tf.nn.l2_loss(output_weights) + 0.001 * tf.nn.l2_loss(hidden_weights) + 0.001 * tf.nn.l2_loss(hidden_weights2) + 0.001 * tf.nn.l2_loss(hidden_weights3))
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output)
  valid_legits1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_legits2 = tf.nn.relu(tf.matmul(valid_legits1, hidden_weights2) + hidden_biases2)
  valid_legits3 = tf.nn.relu(tf.matmul(valid_legits2, hidden_weights3) + hidden_biases3)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_legits3, output_weights) + output_biases)

  test_legits1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_legits2 = tf.nn.relu(tf.matmul(test_legits1, hidden_weights2) + hidden_biases2)
  test_legits3 = tf.nn.relu(tf.matmul(test_legits2, hidden_weights3) + hidden_biases3)
  test_prediction = tf.nn.softmax(tf.matmul(test_legits3, output_weights) + output_biases)


In [98]:
# execute
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 1.0}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.828151
Minibatch accuracy: 7.8%
Validation accuracy: 40.8%
Minibatch loss at step 500: 1.300217
Minibatch accuracy: 90.6%
Validation accuracy: 86.0%
Minibatch loss at step 1000: 0.979367
Minibatch accuracy: 88.3%
Validation accuracy: 87.5%
Minibatch loss at step 1500: 0.889661
Minibatch accuracy: 85.2%
Validation accuracy: 88.2%
Minibatch loss at step 2000: 0.729428
Minibatch accuracy: 88.3%
Validation accuracy: 89.2%
Minibatch loss at step 2500: 0.572219
Minibatch accuracy: 92.2%
Validation accuracy: 88.6%
Minibatch loss at step 3000: 0.726842
Minibatch accuracy: 85.2%
Validation accuracy: 89.7%
Test accuracy: 94.6%


In [ ]:
# NN 2 layers (89.4%): Normal
# NN 2 layers (89.4%): Normal + dropout
# NN 3 layers (89.7%): Normal + dropout